In [5]:
from typing import *
from concurrent.futures.process import ProcessPoolExecutor
import logging
import collections
from datetime import datetime, timedelta
from pathlib import Path
from importlib import reload
import itertools
import functools
import subprocess
import io
import os
import gc
import re
import sys
import time
import logging
import pickle
import json
import random
import string
import numpy as np
import scipy
from scipy import stats
from tqdm import tqdm
import pandas as pd
from sklearn import model_selection, metrics, preprocessing, ensemble, neighbors, cluster, decomposition, inspection, \
    linear_model
import matplotlib.pyplot as plt
from functools import partial
import torch
import torch.nn as nn
from core import *

from fastai.vision.all import *
from slibtk import slibtk

logging.basicConfig(**log_config)
logger = logging.getLogger(__name__)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.5f}'.format)

tqdm.pandas()

In [6]:
def save_img(ser: pd.Series) -> str:
    fig, ax = plt.subplots()
    ax.bar(['value'], ser['value'])
    ax.set_ylim([0, 1])
    img_path = f'{int(ser["index"])}_{ser["value"]}.png'
    plt.savefig((Paths.RAW / img_path).as_posix())
    logger.info(f'saving: {img_path}')
    return img_path


@slibtk.with_cache(Paths.CACHES)
def make_charts(n: int = 1000) -> pd.DataFrame:
    df = pd.Series(np.random.rand(n)).to_frame('value').reset_index()
    df['img_path'] = df.progress_apply(save_img, axis=1)
    return df


def rm_charts() -> None:
    for p in Paths.RAW.glob('*.png'): p.unlink()


def parse_label(fname: str):
    return float(re.search('\d+_(0\.\d+)\.png', fname).group(1))

In [ ]:
df = make_charts()

  0%|          | 0/1000 [00:00<?, ?it/s]08-05-2021 12:14:48 INFO     [<ipython-input-6-99e41beea523>:7] saving: 0_0.6676713744074252.png
08-05-2021 12:14:48 INFO     [<ipython-input-6-99e41beea523>:7] saving: 1_0.2334833337747354.png
  0%|          | 3/1000 [00:00<00:41, 24.12it/s]08-05-2021 12:14:48 INFO     [<ipython-input-6-99e41beea523>:7] saving: 2_0.3153774552704245.png
08-05-2021 12:14:48 INFO     [<ipython-input-6-99e41beea523>:7] saving: 3_0.3126514181904535.png
08-05-2021 12:14:48 INFO     [<ipython-input-6-99e41beea523>:7] saving: 4_0.7616839017909078.png
  1%|          | 6/1000 [00:00<00:42, 23.60it/s]08-05-2021 12:14:49 INFO     [<ipython-input-6-99e41beea523>:7] saving: 5_0.4885857848368841.png
08-05-2021 12:14:49 INFO     [<ipython-input-6-99e41beea523>:7] saving: 6_0.31465369050228.png
08-05-2021 12:14:49 INFO     [<ipython-input-6-99e41beea523>:7] saving: 7_0.7391188001430533.png
  1%|          | 9/1000 [00:00<00:41, 23.70it/s]08-05-2021 12:14:49 INFO     [<ipython-inp

In [ ]:
path = next(Paths.RAW.iterdir())
im = Image.open(path)
im.to_thumb(128, 128)
plt.show()

fns = get_image_files(Paths.RAW)

charts = DataBlock(
    blocks=(ImageBlock, RegressionBlock),
    get_x=ColReader()
)